Import Libraries

In [1]:
import networkx as nx
import csv
from itertools import zip_longest
from lib.xyz import NEURON_POSITIONS

convert two csvs (labels, and adjacency matrix) into a dictionary containing all outgoing connections each node has
- ex: `{label:[1, 27, 105, ...], ...}`

In [2]:
def csvsToDict(labelsLoc, connectionsLoc, type='Matrix', delimit='\t'):
  output = {}
  # open file locations
  with open(labelsLoc) as keys, open(connectionsLoc) as vals:
    # create csv readers to read files
    reader_keys, reader_vals = csv.reader(keys), csv.reader(vals, delimiter=delimit)
    # loop over file row by row
    if (type == 'Matrix'):
      for [key], val in zip_longest(reader_keys, reader_vals):
        # get the list of all nodes which receive incoming edges from the labeled node
        output[key] = [i for i, v in enumerate(val) if int(v) == 1]
    elif (type == 'List'):
      for [k] in reader_keys:
        output[k] = []
      for v in reader_vals:
        node1, node2, type, synapses = v
        if node1 in output and node2 in output:
          output[node1].append((node2, type, synapses))
    return output

Returns a DiGraph where each node is labeled, connected, and has an x, y, and z coordinate

In [4]:
def createGraph(adjDict, type='Matrix'):
  G = nx.DiGraph()

  # create nodes
  # get the labels (k) and the spatial positions (x and y)  
  for k in adjDict.keys():
    x,y,z = NEURON_POSITIONS[k]
    G.add_node(k,x=x,y=y,z=z)

  # create edges
  if (type == 'Matrix'):
    for n in G.nodes():
      for i in adjDict[n]:
        m = list(G.nodes())[i]
        G.add_edge(n,m)
  elif (type == 'List'):
     for n in G.nodes():
      for (m, connectionType, synapseCount) in adjDict[n]:
        G.add_edge(n,m, type=connectionType, synapses=synapseCount)  

  return G

Creates a gexf file given csvs for the labels, adjacency Matrix, and spatial (2D) positions

In [5]:
def convert(folder_outer, connectionsType='Matrix'): #file_positions=None,
  input_labels = f'data/{folder_outer}/source/celegans131labels.csv' # location of file containing labels of each neuron
  input_adjMatrix = f'data/{folder_outer}/source/connections.csv' # location of file containing adjacency matrix

  output = f'data/{folder_outer}/original/xyz.gexf' # output file location

  delimiter = connectionsType == 'Matrix' ? ',' : '\t'
  adjDict = csvsToDict(input_labels, input_adjMatrix, connectionsType, delimit=delimiter) # create adjacency dictionary

  G = createGraph(adjDict, connectionsType) # create graph

  print(len(G.edges()))

  nx.write_gexf(G, output) # save graph as gexf file
  return G

Kaiser 131

In [6]:
convert('Kaiser/131', 'Matrix')

764


Witvliet Datasets

In [7]:
for i in range(1,9):
  convert(f'Witvliet/d{i}', 'List')

487
613
625
736
952
887
1187
1202


White et al (1986) N2U

In [8]:
G_White = convert('white/N2U'r, 'List')

873
Stored 'G_White' (DiGraph)
